In [ ]:
import os
import asyncio
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
os.environ["OPENAI_API_KEY"] =""
os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain import ConversationChain
from langchain.agents import initialize_agent, load_tools, AgentType

In [ ]:
from langchain.chains.

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.prompts import FewShotPromptTemplate

from langchain.document_loaders import HNLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

from langchain.chains import SimpleSequentialChain
from langchain.chains.summarize import load_summarize_chain

In [ ]:
llm = OpenAI(model = 'text-davinci-003',temperature = 0.5)
text = "what do you think about God?"
print(llm.invoke(text))

In [ ]:
def generate_serially(text):
    for _ in range(10):
        output = llm.invoke(text)
        print(output)

async def generate_async(text):
    output = await llm.ainvoke(text)
    print(output)

async def generate_concurrently(text):
    task = [generate_async(text) for _ in range(10)]
    await asyncio.gather(*task)

In [ ]:
#generate_serially(text)
#await generate_concurrently(text)

In [ ]:
prompt = PromptTemplate(input_variables= ["name", "age"],
                        template = "What are the characterics of a someone around the age of {age} living {name}")
print(llm.invoke(prompt.format(name = 'califonia', age = 34)))

In [ ]:
prompt = PromptTemplate(input_variables = ["adjective"],
                        template = "What are some common characteristics of {adjective}")
print(llm.invoke(prompt.format(adjective = "God")))

In [ ]:
prompt = PromptTemplate(input_variables= ["description"],
                        template = "We are a telecommunication company and we are trying to prevent customers from churning\
                            . Generate a promotional mail for one of our customer using information in {description} as \
                                context and it should have a subject relating to the context")
context = "Generate a mail for one of our customers in a relationship informing them of a network upgrade coming to the area soon"
print(llm.invoke(prompt.format(description = context)))

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("fish"))

In [ ]:
tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(tools, llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)
agent.invoke("who is the current leader of Japan? What is the larget prime number that is smaller than the president age")

In [ ]:
conversation = ConversationChain(llm = llm, verbose = True)
conversation.invoke(input = "Hi there")

In [ ]:
conversation.invoke(input = "i'm doinf fine. How are you doing yourself")

In [ ]:
prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = "A {input} if to a {output}"
)

examples = [
    {"input": "pirate", "output": "ship"},
    {"input":"pilot", "output":"plane"}
]

In [ ]:
selector = SemanticSimilarityExampleSelector.from_examples(examples,
                                                           OpenAIEmbeddings(),
                                                           FAISS,
                                                           k=2)

In [ ]:
similar_prompt = FewShotPromptTemplate(
    example_selector=selector,
    example_prompt=prompt,
    prefix="Give the location an item can be found",
    suffix = "input: {noun}\n output:",
    input_variables=['noun']
)

noun = "student"
print(llm.invoke(similar_prompt.format(noun = noun)))

In [ ]:
print(similar_prompt.format(noun = noun))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap = 20
)

In [ ]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(text, embedding = embeddings)
retrieval = db.as_retriever()
embedding_list = embeddings.embed_documents()

In [ ]:
chat_history = ChatMessageHistory()
llm = ChatOpenAI(temperature = 0.5)

user_input = input("Enter your message:",)
for i in range(5):

    chat_history.add_user_message(user_input)
    response = llm(chat_history.messages)
    print(user_input)
    print(response.content)
    chat_history.add_ai_message(response.content)

In [ ]:
history = ChatMessageHistory()
history.add_ai_message()
history.add_user_message()